# Agents in LangChain

Agents are AI systems that can:
- **Reason** about problems
- **Decide** which tools to use
- **Act** by executing tools
- **Observe** results and continue reasoning

We'll use `create_react_agent` from LangGraph - the modern way to build agents.

In [ ]:
# Install required packages
# !pip install langgraph langchain-google-genai

In [6]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from dotenv import load_dotenv

load_dotenv()

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

## 1. Define Tools

Create custom tools that the agent can use.

In [3]:
from datetime import datetime

# Tool 1: Calculator
@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression. Input should be a valid math expression like '2 + 2' or '10 * 5'."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Tool 2: Get Current Date/Time
@tool
def get_current_datetime() -> str:
    """Get the current date and time. Use this when asked about today's date or current time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Tool 3: Weather (Mock)
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city. Input should be a city name."""
    # Mock weather data
    weather_data = {
        "bangalore": "28°C, Partly Cloudy",
        "mumbai": "32°C, Humid",
        "delhi": "22°C, Sunny",
        "new york": "15°C, Rainy",
        "london": "12°C, Cloudy",
    }
    city_lower = city.lower()
    if city_lower in weather_data:
        return f"Weather in {city}: {weather_data[city_lower]}"
    return f"Weather in {city}: 25°C, Clear skies (default)"

# List of tools
tools = [calculator, get_current_datetime, get_weather]
print(f"Created {len(tools)} tools: {[t.name for t in tools]}")

Created 3 tools: ['calculator', 'get_current_datetime', 'get_weather']


## 2. Create the Agent

Use `create_react_agent` to create an agent with tools.

In [8]:
# Create the ReAct Agent
agent = create_react_agent(
    model=llm,
    tools=tools
)

print("✅ Agent created successfully!")

✅ Agent created successfully!


C:\Users\mohsi\AppData\Local\Temp\ipykernel_2736\1000792804.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


## 3. Run the Agent

Invoke the agent with different queries.

In [9]:
# Test 1: Math calculation
response = agent.invoke({
    "messages": [("user", "What is 25 multiplied by 4?")]
})

# Get the final response
print("Query: What is 25 multiplied by 4?")
print(f"Response: {response['messages'][-1].content}")

Query: What is 25 multiplied by 4?
Response: 25 multiplied by 4 is 100.


In [10]:
# Test 2: Get current date/time
response = agent.invoke({
    "messages": [("user", "What is today's date and time?")]
})

print("Query: What is today's date and time?")
print(f"Response: {response['messages'][-1].content}")

Query: What is today's date and time?
Response: Today's date and time is 2026-01-07 17:26:49.


In [11]:
# Test 3: Weather query
response = agent.invoke({
    "messages": [("user", "What's the weather in Bangalore?")]
})

print("Query: What's the weather in Bangalore?")
print(f"Response: {response['messages'][-1].content}")

Query: What's the weather in Bangalore?
Response: The weather in Bangalore is 28°C and partly cloudy.


In [12]:
# Test 4: Complex query (uses multiple tools)
response = agent.invoke({
    "messages": [("user", "What's 100 divided by 5, and also tell me the current time?")]
})

print("Query: What's 100 divided by 5, and also tell me the current time?")
print(f"Response: {response['messages'][-1].content}")

Query: What's 100 divided by 5, and also tell me the current time?
Response: [{'type': 'text', 'text': '100 divided by 5 is 20.0. The current time is 17:27:38 on 2026-01-07.', 'extras': {'signature': 'CqoCAXLI2nykUivACXZuSRT4nvuea43LjRzMiGDBP1wqUBloFw8KV9OzMLtygIfoGk7QS6LIhXv6UYpR+4RKDKIS5/FEvCuSZp2Q3xmunsL8mjtTDkua9fk1GdU8J4LEIq4w9rYQQ2FADMdO4Hd15dNy0hBvV2YFDTpKEtnByavfQbMGEq/kJKiytIvcHQUt4u/ie2e8mtSDGmavFrjDzGD4hNYRKQSqQzO39r0pK+Ard3J9YY3w+Xo1rKysznG1ezrsFFAjB9YheiboUe/mnkF13hyLp3ef82j5NE8p0cVzgRfHmXxEpfT054Vxylx4TQh3oY7tcsshqzMjNU/whiPEZr1S06mkfz6Xictp1FdbHAocITV+USYCuugq0hPFoSJQcSRW73Gizbcsaw=='}}]


## 4. View Agent's Reasoning (Step by Step)

See how the agent thinks and decides which tools to use.

In [13]:
# Stream the agent's steps to see its reasoning
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

response = agent.invoke({
    "messages": [("user", "Calculate 15 * 8 and then tell me the weather in Mumbai")]
})

print("Agent's Full Message History:")
print("=" * 50)
for msg in response['messages']:
    if isinstance(msg, HumanMessage):
        print(f"👤 User: {msg.content}")
    elif isinstance(msg, AIMessage):
        if msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"🤖 Agent calling tool: {tc['name']}({tc['args']})")
        elif msg.content:
            print(f"🤖 Agent: {msg.content}")
    elif isinstance(msg, ToolMessage):
        print(f"🔧 Tool Result: {msg.content}")
    print("-" * 50)

Agent's Full Message History:
👤 User: Calculate 15 * 8 and then tell me the weather in Mumbai
--------------------------------------------------
🤖 Agent calling tool: calculator({'expression': '15 * 8'})
--------------------------------------------------
🔧 Tool Result: Result: 120
--------------------------------------------------
🤖 Agent calling tool: get_weather({'city': 'Mumbai'})
--------------------------------------------------
🔧 Tool Result: Weather in Mumbai: 32°C, Humid
--------------------------------------------------
🤖 Agent: [{'type': 'text', 'text': '15 * 8 is 120. The weather in Mumbai is 32°C, Humid.', 'extras': {'signature': 'CqICAXLI2nw2CRzDhiKuhTVpBCYFd5OdVdYwXNc6vgrbgM0J1vLUQiSg4yzDa5TCwmiT69T7RdrKs+zGgF2nJqAyJSpquuHf3KpWW7TCJ8L177vo8cSQB2XSHktIMXx8GQJHG7iULjt+JXtVPmW5Z/5b2heHgux142EM05TBohXTbBOQWMZ6mhlgq+Z8ddWUCqQVD8b9vZKSlQe5Bs2EHNPtyTzhC6NuyYYBfH9WNqXROPvn/ZNZv+PC9H7v4VXNw17bh5UpQ7b94uToqf4E11nQOeQBoUiGxzxKdk0g1Mmk2FUR/ytXTvt4t5VC1EOAyUm9pXZTTLcBNnMbBCtliuBQyCEkZ

## 5. Agent with System Prompt

Customize the agent's behavior with a system prompt.

In [16]:
# Create agent with custom system prompt
system_prompt = """You are a helpful AI assistant named Jarvis. 
You are friendly, concise, and always use the available tools when needed.
When greeting users, introduce yourself as Jarvis."""

agent_with_prompt = create_react_agent(
    model=llm,
    tools=tools,
    prompt=system_prompt
)

# Test the personalized agent
response = agent_with_prompt.invoke({
    "messages": [("user", "Hello! Who are you and what can you do?")]
})

print(response['messages'][-1].content)

C:\Users\mohsi\AppData\Local\Temp\ipykernel_2736\2427668964.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_with_prompt = create_react_agent(


Hello! I'm Jarvis, a friendly AI assistant. I can help you with a few things:

*   **Calculate**: I can evaluate mathematical expressions for you.
*   **Get the current date and time**: I can tell you what today's date and time are.
*   **Get the weather**: I can fetch the current weather for any city you'd like.


## Summary

| Component | Description |
|-----------|-------------|
| `create_react_agent()` | Creates a ReAct agent with tools |
| `@tool` decorator | Define custom tools |
| `agent.invoke()` | Run the agent with a query |
| `state_modifier` | Add system prompt to customize behavior |

### ReAct Loop:
1. **Reason** - Agent thinks about the query
2. **Act** - Agent decides to use a tool
3. **Observe** - Agent sees the tool result
4. **Repeat** - Until the answer is complete